In [55]:
# Load Pytesseract and other libraries#
from PIL import Image, ImageFilter, ImageEnhance
import pytesseract
import pandas as pd
import numpy as np
pytesseract.pytesseract.tesseract_cmd = r'C:\\Users\\jonat\\Tesseract-OCR\\tesseract.exe'

# Open image and check it works, initialise variables #
filename = "PID with coloured instruments.tif"
PID = Image.open(filename)
PID.show()
img = PID.convert("RGB")
dim = PID.size
height = dim[0]
width = dim[1]
print(dim)

factor = 15

# Sharpness filter 
img2 = img.filter(ImageFilter.SHARPEN)

# Create a contrast enhancer object
contrast_enhancer = ImageEnhance.Contrast(img2)
# Adjust contrast
sharp_img = contrast_enhancer.enhance(factor)
sharp_img.show()

# Stream detector
# Initialise dictionaries
RGBvals = {}
streams = {}
colours = {}
counters = {}
mutefactor = 32

# Load target variables from an excel sheet with colours for streams
target_streams = pd.read_excel('colours.xlsx')
rows = target_streams.shape[0]
# From excel sheet with list of coloured streams, load target colours and index match to colours
for i in range(rows):
    streams[i] = {}
    RGB = []
    for j in range(1,4):
        mute = int(round(target_streams.loc[i][j]/mutefactor))
        RGB.append(mute)
    colours[i] = tuple(RGB)
    counters[i] = 0
colours[2] = (8,0,0)
colours[3] = (0,0,16)
print(colours)

# Find all matching points with the target colour, and input them into a nested dictionary corresponding to each stream
for i in range(height):
    for j in range(width):
        pixel = sharp_img.getpixel((i, j))
        muted = list(pixel)
        for p in range(len(muted)):
            muted[p] = int(round(muted[p]/mutefactor))
        RGBvals[(i, j)] = tuple(muted)
        for k in range(len(colours)):
            if RGBvals[(i,j)] == colours[k]:
                b = counters[k]
                streams[k][b] = tuple((i,j))
                counters[k] += 1



(9362, 6623)
{0: (0, 8, 0), 1: (8, 5, 0), 2: (8, 0, 0), 3: (0, 0, 16)}


In [54]:
# Load Grid Size
gridcol = int(input("How many columns are there in your P&ID grid?"))
gridrow = int(input("How many rows are there in your P&ID grid?"))
boxheight = height/gridrow
boxwidth = width/gridcol

# Function to Convert Letters in Grid Reference to a box number
def letter_to_alphabetical_order(letter):
    if len(letter) != 1 or not letter.isalpha():
        return None  # Invalid input
        
    letter = letter.upper()  # Convert to uppercase
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    
    return alphabet.index(letter) + 1

# Load valve and sensor positions
valvesin = pd.read_excel('Valve list.xlsx')
valves = valvesin.to_numpy()
sensorsin = pd.read_excel('Indicator List.xlsx')
sensors = sensorsin.to_numpy()
print(len(sensors))

# Convert the box positions to a pixel reference
valvecoords = []
sensorcoords = []

for i in range(len(valves)):
    valvecoords.append([0,0,0,0])
    valvecoords[i][0] = valves[i,1]
    valvecoords[i][1] = int(boxheight * (letter_to_alphabetical_order(valves[i,3][0]) + 0.5))
    valvecoords[i][2] = int(boxwidth * (int(valves[i,3][1:]) + 0.5))
    
for j in range(len(sensors)):
    sensorcoords.append([0,0,0,0])
    sensorcoords[j][0] = sensors[j,0]
    sensorcoords[j][1] = int(boxheight * (letter_to_alphabetical_order(sensors[j,3][0]) + 0.5))
    sensorcoords[j][2] = int(boxwidth * (int(sensors[j,3][1:]) + 0.5))

filename2 = "PID with coloured instruments.tif"
file2 = Image.open(filename2)
PIDkey = file2.convert("RGB")
dim2 = file2.size
height2 = dim2[0]
width2 = dim2[1]
print(dim2)
mutefactor2 = 32

# Find all the valves in the image
# Load target variables from an excel sheet with colours for streams - replace with colour chart of transmitters and valves
key = pd.read_excel('Indicator Colour Chart.xlsx')
rowno = key.shape[0]

RGBvals = {}
instruments = {}
instrumentkeys = {}
colours = {}
counters2 = {}

for i in range(rowno):
    instruments[i] = {}
    instrumentkeys[i] = key.loc[i][4]
    RGB = []
    for j in range(1,4):
        mute2 = int(round(key.loc[i][j]/mutefactor2))
        RGB.append(mute2)
    colours[i] = tuple(RGB)
    counters2[i] = 0
print(instrumentkeys,colours,counters2)

# Find all matching points with the target colour, and input them into a nested dictionary corresponding to each stream
for i in range(height):
    for j in range(width):
        pixel = sharp_img.getpixel((i, j))
        muted = list(pixel)
        for p in range(len(muted)):
            muted[p] = int(round(muted[p]/mutefactor))
        RGBvals[(i, j)] = tuple(muted)
        for k in range(len(colours)):
            if RGBvals[(i,j)] == colours[k]:
                b = counters2[k]
                instruments[k][b] = tuple((i,j))
                counters2[k] += 1

print(counters2)

How many columns are there in your P&ID grid? 8
How many rows are there in your P&ID grid? 7


97
(9362, 6623)
{0: 'Manual Valve', 1: 'SOV', 2: 'FCV', 3: 'PCV', 4: 'RV', 5: 'XV', 6: 'FT', 7: 'TT', 8: 'LT', 9: 'PT', 10: 'AT'} {0: (8, 0, 0), 1: (0, 0, 8), 2: (8, 6, 0), 3: (0, 8, 0), 4: (8, 0, 8), 5: (0, 8, 8), 6: (4, 0, 4), 7: (0, 4, 4), 8: (6, 0, 2), 9: (4, 4, 0), 10: (2, 0, 6)} {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}
{0: 57705, 1: 5342, 2: 276, 3: 1312, 4: 22640, 5: 5351, 6: 635, 7: 1049, 8: 0, 9: 353, 10: 0}


In [56]:
print(valvecoords)

[['FCV102', 24742, 13659, 0], ['NRV100a', 24742, 16143, 0], ['NRV100b', 24742, 16143, 0], ['NRV101', 15380, 17799, 0], ['NRV102', 19392, 17799, 0], ['NRV104', 20730, 15315, 0], ['NRV105a', 20730, 12004, 0], ['NRV105b', 20730, 13659, 0], ['NRV106', 18055, 7036, 0], ['NRV107', 19392, 2069, 0], ['NRV108', 19392, 2897, 0], ['NRV109', 22067, 17799, 0], ['NRV110', 20730, 15315, 0], ['NRV111', 15380, 13659, 0], ['NRV400', 8693, 12832, 0], ['NRV401', 6018, 15315, 0], ['PCV103', 24742, 12832, 0], ['PCV104', 19392, 14487, 0], ['PCV115', 19392, 18627, 0], ['PCV116', 15380, 18627, 0], ['PCV117', 22067, 17799, 0], ['PCV119', 15380, 16143, 0], ['PCV120a', 19392, 12004, 0], ['PCV120b', 16717, 12004, 0], ['PCV122', 18055, 8692, 0], ['PCV126', 19392, 4553, 0], ['PCV300', 10030, 15315, 0], ['PCV302', 10030, 7864, 0], ['PCV415', 4681, 12004, 0], ['RV101', 16717, 8692, 0], ['RV102', 19392, 15315, 0], ['RV103', 15380, 18627, 0], ['RV104', 22067, 17799, 0], ['RV105', 15380, 16143, 0], ['RV106a', 18055, 1200

In [ ]:
# TEST COLOURS!
colours[2] = (255,255,0)
colours[3] = (155,0,0)

text_from_image = pytesseract.image_to_data(Image.open("Test2.jpg"))
print(text_from_image)

# Contrast filter
factor = 1
img = PID.convert("RGB")
# Create a contrast enhancer object
contrast_enhancer = ImageEnhance.Contrast(img)
# Adjust contrast
sharp_img = contrast_enhancer.enhance(factor)
sharp_img.show()

# Contrast filter
factor = 3
img = PID.convert("RGB")

# Sharpness filter 
img2 = img.filter(ImageFilter.SHARPEN)

# Create a contrast enhancer object
contrast_enhancer = ImageEnhance.Contrast(img2)
# Adjust contrast
sharp_img = contrast_enhancer.enhance(factor)
sharp_img.show()

# Find all matching points with the target colour, and input them into a nested dictionary corresponding to each stream
for i in range(height):
    for j in range(width):
        pixel = sharp_img.getpixel((i, j))
        RGBvals[(i, j)] = pixel
        for k in range(len(colours)):
            if RGBvals[(i,j)] == colours[k]:
                b = counters[k]
                streams[k][b] = tuple((i,j))
                counters[k] += 1

# Contrast filter
factor = 3

# Sharpness filter 
img2 = img.filter(ImageFilter.SHARPEN)

# Create a contrast enhancer object
contrast_enhancer = ImageEnhance.Contrast(img2)
# Adjust contrast
sharp_img = contrast_enhancer.enhance(factor)
sharp_img.show()

file = sharp_img.save("Test2.jpg")
text_from_image = pytesseract.image_to_data(PID)
print(text_from_image)